In [4]:
import alpaca_trade_api as tradeapi
import concurrent.futures
from datetime import datetime, timedelta
import pandas as pd
from market_data import DataFetcher
import backtest
import talib as ta

# Before writing an algorithm, read this notebook to understand how a backtest is run

In [5]:
# Intializing some backtesting stuff
universe = ['AAPL', 'MMM', 'ABT', 'ABBV', 'ACN', 'ATVI', 'AYI', 'ADBE', 'AMD', 'AAP', 'AES', 'AET', 'AMG', 'AFL']
initial_cash = 30000 # This can be any amount you want. For the sake of this notebook it doesn't really matter
start_date = '2019-1-1'
end_date = '2019-2-12'
algorithm_name = '<your algo>' # Not relevant for this tutorial
bt = backtest.Backtest(algorithm_name, initial_cash, universe, start_date, end_date)

## price_map contains a python dictionary where each key is a stock symbol refering to a pandas DF where each row contains daily data for a day between the start and end date

In [6]:
price_map = bt.set_historical_data()
price_map = bt.universe_data
print (price_map['AAPL'].tail()) # Each symbol refers to a dataframe with basic daily data for each day

INFO:backtest:Fetching Data...
INFO:backtest:Complete.


              open    high       low   close    volume
2019-02-06  174.65  175.57  172.8531  174.24  28239591
2019-02-07  172.40  173.94  170.3400  170.94  31741690
2019-02-08  168.99  170.66  168.4200  170.41  23819966
2019-02-11  171.05  171.21  169.2500  169.43  20993425
2019-02-12  170.10  171.00  169.7000  170.89  22283523


## Below we have a dataframe consisting of daily data for a single day where each row corresponds to a specific stock

# \*\*\* Below we will implement the functions needed for our Strategy

### The functions we need are add_tech_ind(price_map), rank_stocks(daily_data) , stocks_to_buy(portfolio, daily_data) and stocks_to_sell(portfolio, daily_data)


### *add_tech_ind(price_map)* takes the price map that we see printed out above and for each symbol in the price_map you can add techincal indicators which can be used to determine whether or not to buy or sell a stock

In [7]:
# Visit for technical indicator documentation: http://mrjbq7.github.io/ta-lib/funcs.html
def add_tech_ind(price_map):
    new_price_map = {}

    for symbol in price_map:
        symbol_data = price_map[symbol]
        symbol_data['ema_50'] = ta.EMA(symbol_data['open'], timeperiod=50)
        symbol_data['ema_open_pct_diff'] = (symbol_data['ema_50'] - symbol_data['open']) / symbol_data['open']
        new_price_map[symbol] = symbol_data

    return new_price_map

## Below is what our new price_map looks like with our new columns

In [8]:
new_price_map = add_tech_ind(price_map)
print (new_price_map['AAPL'].tail())

              open    high       low   close    volume      ema_50  \
2019-02-06  174.65  175.57  172.8531  174.24  28239591  166.660432   
2019-02-07  172.40  173.94  170.3400  170.94  31741690  166.885513   
2019-02-08  168.99  170.66  168.4200  170.41  23819966  166.968042   
2019-02-11  171.05  171.21  169.2500  169.43  20993425  167.128119   
2019-02-12  170.10  171.00  169.7000  170.89  22283523  167.244663   

            ema_open_pct_diff  
2019-02-06          -0.045746  
2019-02-07          -0.031987  
2019-02-08          -0.011965  
2019-02-11          -0.022928  
2019-02-12          -0.016786  


### This is what the daily data looks like for an arbitrary day that gets fed into stocks_to_buy(), and stocks_to_sell()

In [9]:
bt.universe_data = new_price_map
daily_data_dict = bt.daily_data_dict()
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d').date() + timedelta(days=1)
print(daily_data_dict[start_date_dt])

         close      ema_50  ema_open_pct_diff    high       low    open  \
symbol                                                                    
ACN     140.59  154.766376           0.113988  141.19  138.2800  138.93   
AAPL    157.92  179.925147           0.161632  158.85  154.2300  154.89   
MMM     190.95  196.816707           0.047901  190.99  186.7000  187.82   
ABBV     89.23   88.262546          -0.032633   91.26   88.4200   91.24   
ATVI     47.03   54.392940           0.202054   47.51   45.1367   45.25   
ABT      69.50   70.002553          -0.005504   70.96   69.0700   70.39   
AYI     115.23  121.655246           0.075548  116.18  111.7100  113.11   
ADBE    224.57  236.760889           0.076626  226.17  219.0000  219.91   
AMD      18.83   20.350917           0.129979   19.00   17.9800   18.01   
AES      14.18   14.906285           0.044589   14.31   14.1000   14.27   
AAP     157.92  164.619063           0.054169  159.92  153.8200  156.16   
AMG      98.23  108.35102

### When the backtester runs each day it will feed data into stocks_to_buy(), and stocks_to_sell(). rank_stocks() is a helper function that stocks_to_buy(), and stocks_to_sell() calls to help determine which stocks to buy and which stocks to sell

In our rank_stocks() function below, we rank the stocks in descending order by the ema_open_pct_diff coloumn.

In [10]:
def rank_stocks(daily_data):
    sorted_daily_data = daily_data.sort_values('ema_open_pct_diff', ascending=False)
    return sorted_daily_data

### Below you will notice that both functions take a parameter called portfolio. Each day of the backtest, the backtester will send this to your functions to tell your algo which stocks you own already

Ex: portfolio = ['AAPL', 'MMM', 'ABT', 'ABBV', 'ACN']

stocks_to_buy() calls rank_stocks() and buys the top 5 ranked stocks, or if we already own one of those stocks it holds it.

stocks_to_sell() calls rank_stocks() and if the stocks in our portfolio aren't in the top 5 returned from rank_stocks() it sells them.



In [11]:
def stocks_to_buy(portfolio, daily_data):
    # Return an array of tuples with symbol and buy price
    # [('AAPL', 172.00), (BBUY, 30.00)]

    ''' Retrieve the top five stocks with the largest pct change between its open price and 50 day EMA '''
    sorted_daily_data = rank_stocks(daily_data)
    symbols = sorted_daily_data.head(5).index

    buy_orders = []
    for symbol in symbols:
        ''' The buy price is the open price because this algo checks to buy in the morning '''
        if symbol not in portfolio:
            order_tup = (symbol, sorted_daily_data.loc[symbol, 'open'])
            buy_orders.append(order_tup)

    return buy_orders

def stocks_to_sell(portfolio, daily_data):
    # Return an array of tuples with symbol and sell price
    # [('AAPL', 172.00), (BBUY, 30.00)]
    sorted_daily_data = rank_stocks(daily_data)
    buy_symbols = sorted_daily_data.head(5).index

    sell_orders = []
    for symbol in portfolio:
        if symbol not in buy_symbols:
            order_tup = (symbol, sorted_daily_data.loc[symbol, 'open'])
            sell_orders.append(order_tup)

    return sell_orders

### Below we will run a test to make sure our functions work properly. We will start with an arbitrary portfolio and daily data for a random day

Our stocks_to_buy() function returns three new stocks to buy and the prices that we will buy them at

In [2]:
portfolio = ['AAPL', 'MMM', 'ABT', 'ABBV', 'ACN']
daily_data = daily_data_dict[start_date_dt]
sell_stocks = stocks_to_sell(portfolio, daily_data)
print(sell_stocks)

NameError: name 'daily_data_dict' is not defined

In [3]:
new_portfolio = ['AAPL', 'ACN']# Manually updated here, but backtester will update automatically
buy_stocks = stocks_to_buy(new_portfolio, daily_data)
print(buy_stocks)

NameError: name 'stocks_to_buy' is not defined

## We have no errors during testing, so now you can go ahead and make a class and add these functions to it and run a real backtest!